In [2]:
import numpy as np
import pandas as pd

In [3]:
rating_df = pd.read_csv('D:\\Jupyter notebook\\ratings_small.csv')
rating_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


看看這個資料有多大

In [76]:
rating_df.shape

(100004, 4)

看看這個資料有多少不同的值

In [85]:
rating_df.nunique

<bound method DataFrame.nunique of        userId  movieId  rating   timestamp
52635     383       21     3.0   789652009
52641     383       47     5.0   789652009
52684     383     1079     3.0   789652009
56907     409       21     5.0   828212412
56909     409       25     4.0   828212412
...       ...      ...     ...         ...
35125     251    56587     5.0  1476623131
35172     251   134853     5.0  1476623217
35140     251    81847     4.5  1476623282
35096     251     6377     4.5  1476623300
35069     251     1704     4.5  1476640644

[100004 rows x 4 columns]>

切分成訓練集跟測試集
排序前面當訓練集，排序後面當測試集
timestamp是用時間排序的意思

In [86]:
rating_df = rating_df.sort_values('timestamp')
rating_df.head()

,userId,movieId,rating,timestamp
52635,383,21,3.0,789652009
52641,383,47,5.0,789652009
52684,383,1079,3.0,789652009
56917,409,85,5.0,828212412
56913,409,35,4.0,828212412


建造使用者對電影的評分矩陣
LabelEncoder一個list裡面不同的值對應到不同的連續整數

In [92]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

user_ids = user_encoder.fit_transform(rating_df.userId)
movie_ids = movie_encoder.fit_transform(rating_df.movieId)

將總共的數據取8成當作訓練的數據
num_train訓練數量

In [93]:
num_train = int(len(user_ids) * 0.8)
num_train

80003

In [94]:
train_user_ids = user_ids[:num_train]
train_movie_ids = movie_ids[:num_train]
train_ratings = rating_df.rating.values[:num_train]
val_user_ids = user_ids[num_train:]
val_movie_ids = movie_ids[num_train:]
val_ratings = rating_df.rating.values[num_train:]
train_user_ids.shape,train_movie_ids.shape,train_ratings.shape,val_user_ids.shape, val_movie_ids.shape, val_ratings.shape,

((80003,), (80003,), (80003,), (20001,), (20001,), (20001,))

製作使用者跟電影的評分矩陣user2movie
num_users = user_ids.max() + 1=>總使用者數量
num_movies = movie_ids.max() + 1=>總電影數量

In [95]:
num_users = user_ids.max() + 1
num_movies = movie_ids.max() + 1
user2movie = np.zeros([num_users, num_movies])
user2movie[train_user_ids, train_movie_ids] = train_ratings
user2movie

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

compute_user_similarity 計算使用者間相似度
pearson_correlation(x,y) x跟y分別是使用者對於兩個不同電影的偏好向量

In [96]:
def compute_user_similarity_matrix(user2movie):
    similarity_matrix = np.zeros([num_users, num_users])
    for i in range(num_users):
        for j in range(i, num_users):
            
            corr = pearson_correlation(user2movie[i], user2movie[j])
            
            similarity_matrix[i,j] = corr
            similarity_matrix[j,i] = corr
        
        return similarity_matrix
    
def pearson_correlation(x,y):
    
    filt = (x!=0) * (y!=0)
    
    x_mean = x.sum() / x[x!=0].shape
    y_mean = y.sum() / y[y!=0].shape
    
    x = x[filt]
    y = y[filt]
    
    corr = np.sum((x - x_mean) * (y - y_mean)) / (np.sum((y-y_mean) **2) * np.sum((x-x_mean) ** 2)) **0.5
    
    return corr
similarity_matrix = compute_user_similarity_matrix(user2movie)
similarity_matrix[:10]

C:\Users\User\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\User\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


array([[ 1., nan, nan, ...,  1., nan, -1.],
       [nan,  0.,  0., ...,  0.,  0.,  0.],
       [nan,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [nan,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [nan,  0.,  0., ...,  0.,  0.,  0.]])

把NAN去除

In [98]:
similarity_matrix[np.arange(num_users), np.arange(num_users)] = 0
similarity_matrix[np.isnan(similarity_matrix)] = 0
similarity_matrix[:10]

array([[ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        , -1.        ],
       [ 0.        ,  0.        , -0.01606095, ..., -0.95740201,
        -0.49446359,  0.57048961],
       [ 0.        , -0.01606095,  0.        , ...,  0.97327676,
        -0.08336072,  0.52261992],
       ...,
       [ 0.        ,  0.3103714 ,  0.13950249, ..., -0.44308725,
         0.4618251 ,  0.08969916],
       [ 1.        ,  0.1755357 , -0.37364627, ...,  1.        ,
         0.42145455,  0.40640584],
       [ 0.        , -0.16227767,  0.59686456, ..., -0.37286939,
        -0.02181995,  0.50435832]])

user base協同過濾

expand_dims 把一維變二維

In [99]:
def compute_ucf(user2movie, similarity_matrix):
    #yk
    mean_ratings = np.sum(user2movie, axis = 1) / (user2movie !=0).sum(axis =1)
    
    #ykj - yk, user2movie: (num_users, num_movies), mean_ratings: (num_users) -> (num_users, 1)
    user2movie_diff = user2movie - np.expand_dims(mean_ratings, axis =1)
    
    sim_sum = np.sum(np.abs(similarity_matrix), axis =1)
    
    user2movie_diff[user2movie == 0] = 0
    
    #similarity_matrix: (num_users, num_users), user2movie_diff: (num_users, num_movies)
    # -> (num_users, num_movies), sim_sum: (num_users) -> (num_users, 1)
    weighted_sum = np.matmul(similarity_matrix, user2movie_diff) / np.expand_dims(sim_sum, axis=1)
    
    #weighted_sum: (num_users, num_movies), mean_ratings: (num_users)
    scores = weighted_sum + np.expand_dims(mean_ratings, axis=1)
    
    return scores

predictions = compute_ucf(user2movie, similarity_matrix)
predictions[:10]
    

C:\Users\User\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


array([[2.60394488, 2.52142328, 2.53811983, ..., 2.55      , 2.55      ,
        2.55      ],
       [3.51715353, 3.4744639 , 3.4901902 , ..., 3.48684211, 3.48684211,
        3.48684211],
       [3.56477871, 3.56355235, 3.55132752, ..., 3.56862745, 3.56862745,
        3.56862745],
       ...,
       [3.93543738, 3.86472265, 3.83789392, ..., 3.86637931, 3.86637931,
        3.86637931],
       [3.86998992, 3.73762887, 3.72502154, ..., 3.75555556, 3.75555556,
        3.75555556],
       [3.79613736, 3.6997219 , 3.66718739, ..., 3.69565217, 3.69565217,
        3.69565217]])

算出DCG才可實做NDCG衡量

In [100]:
def dcg_at_k(r,k):
    r= r[:k]
    dcg = np.sum(r / np.log2(np.arange(2, len(r) + 2)))
    
    return dcg

reverse= True由大到小

In [101]:
def ndcg_at_k(r,k):
    dcg_max = dcg_at_k(sorted(r, reverse= True),k)
    
    return dcg_at_k(r,k) / dcg_max

In [102]:
def evaluate_prediction(predictions):
    
    ndcgs = []
    
    for target_user in np.unique(val_user_ids):
        
        target_val_movie_ids = val_movie_ids[val_user_ids == target_user]
        target_val_ratings = val_ratings[val_user_ids == target_user]
        
        ndcg = ndcg_at_k(target_val_ratings[np.argsort(-predictions[target_user][target_val_movie_ids])], k=30)
        ndcgs.append(ndcg)
    ndcg = np.mean(ndcgs)
    return ndcg
evaluate_prediction(predictions)

0.8326360059603118

基於物品協同過濾

In [103]:
def compute_item_similarity_matrix(user2movie):
    
    x_mean = user2movie.sum(axis=0) / (user2movie !=0).sum(axis=0)
    
    filt = user2movie == 0
    rating_diff = user2movie - np.expand_dims(x_mean, axis =0)
    
    rating_diff[filt] =0
    
    #(num_movies, num_movies)
    corr = np.matmul(rating_diff.T, rating_diff) / \
    (np.matmul(rating_diff.T **2, (rating_diff !=0)) * np.matmul(rating_diff.T **2, (rating_diff !=0)).T) ** 0.5
    
    return corr
similarity_matrix = compute_item_similarity_matrix(user2movie)
similarity_matrix[:10]

C:\Users\User\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


array([[ 1.        ,  0.32949699,  0.14883085, ...,         nan,
                nan,         nan],
       [ 0.32949699,  1.        ,  0.11771157, ...,         nan,
                nan,         nan],
       [ 0.14883085,  0.11771157,  1.        , ...,         nan,
                nan,         nan],
       ...,
       [ 0.75689798,  0.79614979,  0.71287669, ...,         nan,
                nan,         nan],
       [-0.21912884,  0.5874723 ,  0.34578862, ...,         nan,
                nan,         nan],
       [ 0.02433877,  0.43192313, -0.08695837, ...,         nan,
                nan,         nan]])

In [104]:
similarity_matrix[np.arange(num_movies), np.arange(num_movies)] = 0
similarity_matrix[np.isnan(similarity_matrix)] =0
similarity_matrix[:10]

array([[ 0.        ,  0.32949699,  0.14883085, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32949699,  0.        ,  0.11771157, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.14883085,  0.11771157,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.75689798,  0.79614979,  0.71287669, ...,  0.        ,
         0.        ,  0.        ],
       [-0.21912884,  0.5874723 ,  0.34578862, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.02433877,  0.43192313, -0.08695837, ...,  0.        ,
         0.        ,  0.        ]])

In [105]:
def compute_icf(user2movie, similarity_matrix):
    
    #yk
    mean_ratings = np.sum(user2movie, axis = 0) / (user2movie !=0).sum(axis =0)
    
    #ykj - yk, user2movie: (num_users, num_movies), mean_ratings: (num_users) -> (num_users, 1)
    user2movie_diff = user2movie - np.expand_dims(mean_ratings, axis =0)
    
    sim_sum = np.sum(np.abs(similarity_matrix), axis =1)
    
    user2movie_diff[user2movie == 0] = 0
    
    #similarity_matrix: (num_movies, num_movies), user2movie_diff: (num_movies, num_users)
    # -> (num_users, num_movies), sim_sum: (num_users) -> (num_users, 1)
    weighted_sum = np.matmul(user2movie_diff, similarity_matrix) / np.expand_dims(sim_sum, axis=0)
    
    #weighted_sum: (num_users, num_movies), mean_ratings: (num_users)
    scores = weighted_sum + np.expand_dims(mean_ratings, axis=0)
    
    return scores

predictions = compute_icf(user2movie, similarity_matrix)
predictions[:10]

C:\Users\User\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
C:\Users\User\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


array([[3.91995938, 3.36217492, 3.19289318, ...,        nan,        nan,
               nan],
       [3.92128976, 3.3640285 , 3.19388683, ...,        nan,        nan,
               nan],
       [3.91963665, 3.36367295, 3.19216674, ...,        nan,        nan,
               nan],
       ...,
       [3.92075554, 3.36605949, 3.1937442 , ...,        nan,        nan,
               nan],
       [3.92067407, 3.36313627, 3.1926433 , ...,        nan,        nan,
               nan],
       [3.92048034, 3.36408442, 3.19267732, ...,        nan,        nan,
               nan]])

In [106]:
evaluate_prediction(predictions)

0.872177812234131